<a href="https://colab.research.google.com/github/ldsAS/Tibame-AI-Learning/blob/main/Tibame20250617_%E4%BD%BF%E7%94%A8%E7%B7%9A%E6%80%A7%E5%9B%9E%E6%AD%B8%E6%A8%A1%E5%9E%8B%E9%87%9D%E5%B0%8D%E4%BC%81%E9%B5%9D%E9%AB%94%E9%87%8D%E9%A0%90%E6%B8%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import os
from google.cloud import bigquery

# 設置 Google Cloud 認證 # 此為示範，實際上應放在環境變數裡面(要隱藏起來)
os.environ['GOOGLE_APPLICATION_CREDENTIALS']=r"/content/tibame-gad251-31-0613-c339499c7e0e.json"


# 初始化 BigQuery 客戶端
client = bigquery.Client()
print('connection done')


connection done


In [1]:
us_dataset_id = 'tibame_gad251_31_dataset_us' #定義us資料集
eu_dataset_id = 'tibame_gad251_31_dataset_eu' #定義eu資料集

In [6]:
# 定義 SQL 查詢以創建模型
query = f"""
CREATE OR REPLACE MODEL `{us_dataset_id}.penguins_model`
OPTIONS
  (model_type='linear_reg',
  input_label_cols=['body_mass_g']) AS
SELECT
  *
FROM
  `bigquery-public-data.ml_datasets.penguins`
WHERE
  body_mass_g IS NOT NULL;
"""

# 執行 SQL 查詢以創建模型
query_job = client.query(query)
query_job.result()  # 等待查詢完成

print("線性回歸 model import BigQuery ML")


線性回師 model import BigQuery ML
